In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
alsfrs_o_3mo = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/alsfrs_total_3mo_meta_slope.csv')
alsfrs_r_3mo = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/alsfrs_r_total_3mo_meta_slope.csv')

In [4]:
alsfrs_o_3mo = alsfrs_o_3mo.sort_values(by = 'SubjectID', ascending = True)
alsfrs_o_3mo = alsfrs_o_3mo.reset_index(drop=True)
alsfrs_r_3mo = alsfrs_r_3mo.sort_values(by = 'SubjectID', ascending = True)
alsfrs_r_3mo = alsfrs_r_3mo.reset_index(drop=True)

### Checking the number of NaNs; alsfrs_original vs alsfrs_revised

In [5]:
orig_filtered_nan = alsfrs_o_3mo[alsfrs_o_3mo['alsfrs_total_slope'].isnull()]
orig_filtered_nan.sort_values(by = 'SubjectID', ascending = True)

,SubjectID,alsfrs_total_slope,mean_alsfrs_total,min_alsfrs_total,max_alsfrs_total,n,first,last,interval
3,586,NaN,31.0,31,31.0,1,3,3,0
9,1309,NaN,26.0,26,26.0,1,0,0,0
11,2416,NaN,20.0,20,20.0,1,2,2,0
14,2598,NaN,36.0,36,36.0,1,0,0,0
35,5773,NaN,36.0,36,36.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
6423,988936,NaN,30.0,30,30.0,1,0,0,0
6458,993488,NaN,27.0,27,27.0,1,10,10,0
6491,998019,NaN,16.0,16,16.0,1,13,13,0
6493,998261,NaN,26.0,26,26.0,1,0,0,0


In [6]:
rev_filtered_nan = alsfrs_r_3mo[alsfrs_r_3mo['alsfrs_r_total_slope'].isnull()]
rev_filtered_nan.sort_values(by = 'SubjectID', ascending = True)

,SubjectID,alsfrs_r_total_slope,mean_alsfrs_r_total,min_alsfrs_r_total,max_alsfrs_r_total,n,first,last,interval
2,586,NaN,39.0,39,39,1,3,3,0
5,1309,NaN,34.0,34,34,1,0,0,0
7,2416,NaN,27.0,27,27,1,2,2,0
8,2598,NaN,44.0,44,44,1,0,0,0
16,5773,NaN,42.0,42,42,1,0,0,0
...,...,...,...,...,...,...,...,...,...
3314,974155,NaN,46.0,46,46,1,0,0,0
3350,985752,NaN,40.0,40,40,1,0,0,0
3399,998019,NaN,21.0,21,21,1,13,13,0
3400,998261,NaN,33.0,33,33,1,0,0,0


In [7]:
print("There are", len(orig_filtered_nan["SubjectID"].unique()), "subjects with nans in alsfrs_orig_data.")
print("There are", len(rev_filtered_nan["SubjectID"].unique()), "subjects with nans in alsfrs_rev_data.")

There are 340 subjects with nans in alsfrs_orig_data.
There are 288 subjects with nans in alsfrs_rev_data.


In [8]:
# assigning subjects with nan to a list
extent = list(orig_filtered_nan["SubjectID"].unique())

In [9]:
rev_in_extent = alsfrs_r_3mo.query("SubjectID == {0}".format(extent))
rev_in_extent.sort_values(by = 'SubjectID', ascending = True)

,SubjectID,alsfrs_r_total_slope,mean_alsfrs_r_total,min_alsfrs_r_total,max_alsfrs_r_total,n,first,last,interval
2,586,NaN,39.0,39,39,1,3,3,0
5,1309,NaN,34.0,34,34,1,0,0,0
7,2416,NaN,27.0,27,27,1,2,2,0
8,2598,NaN,44.0,44,44,1,0,0,0
16,5773,NaN,42.0,42,42,1,0,0,0
...,...,...,...,...,...,...,...,...,...
3314,974155,NaN,46.0,46,46,1,0,0,0
3350,985752,NaN,40.0,40,40,1,0,0,0
3399,998019,NaN,21.0,21,21,1,13,13,0
3400,998261,NaN,33.0,33,33,1,0,0,0


In [10]:
# Checking if rev_filtered_nan and rev_in_extent are identical
df = pd.concat([rev_filtered_nan,rev_in_extent])
df = df.reset_index(drop=True) # 인덱스 초기화
 
df_grp = df.groupby(df.columns.tolist()) # 전체 열 비교
df_di = df_grp.groups # 딕셔너리로 만들기 
 
idx = [x[0] for x in df_di.values() if len(x) == 1] # 인덱스 검토
df.loc[idx,:] # Same as df.reindex(idx)

,SubjectID,alsfrs_r_total_slope,mean_alsfrs_r_total,min_alsfrs_r_total,max_alsfrs_r_total,n,first,last,interval


In [11]:
print("Conclusion: There are no data useful to fill nans.")

Conclusion: There are no data useful to fill nans.


### Comparing SubjectIDs; to check if there is any subjects to be added

In [25]:
df1 = alsfrs_o_3mo[['SubjectID']]
orig_subj = list(df1["SubjectID"].unique())
df1

,SubjectID
0,329
1,348
2,533
3,586
4,649
...,...
6502,999166
6503,999482
6504,999823
6505,999880


In [31]:
df2 = alsfrs_r_3mo[['SubjectID']]
df2

,SubjectID
0,348
1,533
2,586
3,902
4,1064
...,...
3401,998542
3402,998756
3403,999166
3404,999823


In [32]:
df2[~df2['SubjectID'].isin(orig_subj)]

,SubjectID


In [34]:
df_v = df1.merge(df2, on='SubjectID', how = 'outer')
df_v

,SubjectID
0,329
1,348
2,533
3,586
4,649
...,...
6502,999166
6503,999482
6504,999823
6505,999880


In [35]:
print("Conclusion: original data completely contains every subjects in revised data.")

Conclusion: original data completely contains every subjects in revised data.
